In [1]:
#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys

In [2]:
# connect to the sim 
client = carla.Client('localhost', 2000)

client.get_available_maps()

# Load layered map for Town 01 with minimum layout plus buildings and parked vehicles
client.load_world('Town04_Opt')
world = client.get_world()

# Improve the weather
weather = carla.WeatherParameters(
    cloudiness=0.0,
    precipitation=0.0,
    sun_altitude_angle=45.0)
world.set_weather(weather)

In [10]:
# Toggle all buildings off
world.unload_map_layer(carla.MapLayer.All)
world.load_map_layer(carla.MapLayer.Ground)

In [4]:
# # get the transform of the player
# player = world.get_spectator()

# # get the waypoint near to the player
# waypoint_A = world.get_map().get_waypoint(player.get_location())

# # display the waypoint
# world.debug.draw_string(waypoint_A.transform.location, 'O', draw_shadow=False,
#                            color=carla.Color(r=255, g=0, b=0), life_time=1000,
#                            persistent_lines=True)



# # get all the next 200 waypoints to waypoint_A and display them
# waypoints = [waypoint_A]
# for _ in range(2000):
#     waypoints.append(waypoints[-1].next(1.0)[0])  # get the next waypoint 1 meter ahead

# for waypoint in waypoints:
#     world.debug.draw_string(waypoint.transform.location, 'O', draw_shadow=False,
#                            color=carla.Color(r=0, g=255, b=0), life_time=1000,
#                            persistent_lines=True)

# # Save the list of waypoints to a file, location and transformation
# with open('waypoints_back.txt', 'w') as waypoint_file:
#     for waypoint in waypoints:
#         waypoint_file.write(str(waypoint.transform.location.x) + ' ' + str(waypoint.transform.location.y) + ' ' + str(waypoint.transform.location.z) + ' ' + str(waypoint.transform.rotation.pitch) + ' ' + str(waypoint.transform.rotation.yaw) + ' ' + str(waypoint.transform.rotation.roll) + '\n')


In [5]:
# Read the file of waypoints waypoints_1.txt, store them in a list and display them
waypoint_path = 'waypoints.txt'

waypoints = []
with open(waypoint_path, 'r') as waypoint_file:
    for line in waypoint_file:
        x, y, z, pitch, yaw, roll = line.split()
        waypoints.append(carla.Transform(carla.Location(float(x), float(y), float(z)), carla.Rotation(float(pitch), float(yaw), float(roll))))

# Display the waypoints
for waypoint in waypoints:
    world.debug.draw_string(waypoint.location, '^', draw_shadow=False,
                           color=carla.Color(r=0, g=255, b=0), life_time=1000,
                           persistent_lines=True)
    

In [6]:
# Move the spectator to the first waypoint
player = world.get_spectator()
player.set_transform(waypoints[0])

# Spawn a racing car in waypoint_A
blueprint_library = world.get_blueprint_library()
bp = blueprint_library.filter('model3')[0]
spawn_point = waypoints[0]

spawn_point.location.z += 1.0

vehicle = world.spawn_actor(bp, spawn_point)



In [11]:
#camera mount offset on the car - you can tweak these to have the car in view or not
CAMERA_POS_Z = 1.5 
CAMERA_POS_X = 0 

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

In [12]:
vehicle.set_autopilot(True)

idx = 0
while True:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    
    # Control the vehicle using computer vision to predict the steering angle
    
    # Save the image as a file with the steering angle
    steering_angle = vehicle.get_control().steer
    cv2.imwrite('data/images/' + str(idx) + '.png',image)

    # Save the steering angle in a json file for all the images names and steering angles, as a dictionary, with images and labels
    with open('data/labels.json', 'a') as labels_file:
        labels_file.write('{"image": "' + str(idx) + '.png", "label": ' + str(steering_angle) + '}\n')

    # Display the image
    cv2.imshow('RGB Camera',image)

    idx += 1

#clean up
cv2.destroyAllWindows()
camera.stop()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
    

In [10]:
# clean vehicle
vehicle.destroy()

True